In [19]:
import pandas as pd
import numpy as np
import random
import argparse
import torch
from torch import nn
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
train = pd.read_parquet(r'/home/coenraadmiddel/Documents/RossmannStoreSales/TabNet/tabnet/train_processed.parquet')
print(train.shape)

SyntaxError: invalid syntax (3042183125.py, line 1)

In [1]:
#select only a couple of columns

#This was done to figure out why there is a categorical dim issue when training. (Because of missing code, see notes.)

train = train[['Store','DayOfWeek'
               ,'Date','Sales','Customers','Open','Promo','StateHoliday'
               ,'SchoolHoliday','StoreType','Assortment','CompetitionDistance','CompetitionOpenSinceMonth'
               ,'CompetitionOpenSinceYear','Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval']]

NameError: name 'train' is not defined

In [ ]:
random.seed(42)
if "Set" not in train.columns:
    train.reset_index(inplace=True, drop=True)
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# check distribution of sales in train set
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
g1 = sns.distplot(train['Sales'],hist = True,label='skewness:{:.2f}'.format(train['Sales'].skew()),ax = ax1)
g1.legend()
g1.set(xlabel = 'Sales', 

ylabel = 'Density', title = 'Sales Distribution')
g2 = sns.distplot(np.log1p(train['Sales']),hist = True,label='skewness:{:.2f}'.format(np.log1p(train['Sales']).skew()),ax=ax2)
g2.legend()
g2.set(xlabel = 'log(Sales+1)',ylabel = 'Density', title = 'log(Sales+1) Distribution')
plt.show()

In [ ]:
    
categorical_columns = ['Store',
                        'DayOfWeek',
                        'Promo',
                        'StateHoliday',
                        'SchoolHoliday',
                        'StoreType',
                        'Assortment',
                        # 'Year',
                        # 'Month',
                        # 'Day',
                        # 'WeekOfYear',
                        'IsPromoMonth']

In [ ]:
train_indices

In [ ]:
from data_openml import data_split

# split x and y
X_all, y_all = train.drop(columns = ['Sales', 'Set']), np.log1p(train[['Sales']].values)

temp = X_all.fillna("MissingValue")
nan_mask = temp.ne("MissingValue").astype(int)

X_train_d, y_train_d = data_split(X_all, y_all, nan_mask, train_indices)
X_valid_d, y_valid_d = data_split(X_all, y_all, nan_mask, valid_indices)
X_test_d, y_test_d = data_split(X_all, y_all, nan_mask, test_indices)


X_train = X_train_d['data']
X_test = X_test_d['data']
X_valid = X_valid_d['data']

y_train = y_train_d['data']
y_test = y_test_d['data']
y_valid = y_valid_d['data']



# X_train = X_all.values[train_indices]
# y_train = y_all.values[train_indices].reshape(-1, 1)

# X_valid = X_all.values[valid_indices]
# y_valid = y_all.values[valid_indices].reshape(-1, 1)

# X_test = X_all.values[test_indices]
# y_test = y_all.values[test_indices].reshape(-1, 1)

In [ ]:
print(type(y_train))
print(type(y_train_d))
print(type(X_train))
print(type(X_train_d))

In [ ]:
2, 2, 2, 2, 2]
cat_idxs = [0, 1, 2, 3, 4, 5, 6, 16]
cat_dims = [1115, 7, 2, 4, 2, 4, 3, 2]

In [ ]:
from models import SAINT
import torch
torch.manual_seed(42)
import wandb


In [ ]:

wandb.init(project="saint_v2_all_rossmann", group = "first" ,name = f'regression_colrow_rossmann_seed42')

In [ ]:
import sys
sys.argv = ['']
parser = argparse.ArgumentParser()

# parser.add_argument('--dset_id', required=True, type=int)
# parser.add_argument('--vision_dset', action = 'store_true')
# parser.add_argument('--task', required=True, type=str,choices = ['binary','multiclass','regression'])
parser.add_argument('--cont_embeddings', default='MLP', type=str,choices = ['MLP','Noemb','pos_singleMLP'])
parser.add_argument('--embedding_size', default=32, type=int)
parser.add_argument('--transformer_depth', default=6, type=int)
parser.add_argument('--attention_heads', default=8, type=int)
parser.add_argument('--attention_dropout', default=0.1, type=float)
parser.add_argument('--ff_dropout', default=0.1, type=float)
parser.add_argument('--attentiontype', default='colrow', type=str,choices = ['col','colrow','row','justmlp','attn','attnmlp'])

parser.add_argument('--optimizer', default='AdamW', type=str,choices = ['AdamW','Adam','SGD'])
parser.add_argument('--scheduler', default='cosine', type=str,choices = ['cosine','linear'])

parser.add_argument('--lr', default=0.0001, type=float)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--batchsize', default=256, type=int)
parser.add_argument('--savemodelroot', default='./bestmodels', type=str)
parser.add_argument('--run_name', default='testrun', type=str)
parser.add_argument('--set_seed', default= 1 , type=int)
parser.add_argument('--dset_seed', default= 5 , type=int)
parser.add_argument('--active_log', action = 'store_true')

parser.add_argument('--pretrain', action = 'store_true')
parser.add_argument('--pretrain_epochs', default=50, type=int)
parser.add_argument('--pt_tasks', default=['contrastive','denoising'], type=str,nargs='*',choices = ['contrastive','contrastive_sim','denoising'])
parser.add_argument('--pt_aug', default=[], type=str,nargs='*',choices = ['mixup','cutmix'])
parser.add_argument('--pt_aug_lam', default=0.1, type=float)
parser.add_argument('--mixup_lam', default=0.3, type=float)

parser.add_argument('--train_mask_prob', default=0, type=float)
parser.add_argument('--mask_prob', default=0, type=float)

parser.add_argument('--ssl_avail_y', default= 0, type=int)
parser.add_argument('--pt_projhead_style', default='diff', type=str,choices = ['diff','same','nohead'])
parser.add_argument('--nce_temp', default=0.7, type=float)

parser.add_argument('--lam0', default=0.5, type=float)
parser.add_argument('--lam1', default=10, type=float)
parser.add_argument('--lam2', default=1, type=float)
parser.add_argument('--lam3', default=10, type=float)
parser.add_argument('--final_mlp_style', default='sep', type=str,choices = ['common','sep'])


opt = parser.parse_args()

In [ ]:
opt.attention_heads = min(4, opt.attention_heads)
opt.attention_dropout = 0.8
opt.embedding_size = min(32, opt.embedding_size)
opt.ff_dropout = 0.8

In [ ]:
wandb.config.update(opt)
#for regression this is the output dimension
= 1


In [ ]:
= 'cuda' if torch.cuda.is_available() else 'cpu'

print("Using {}".format(device))

In [ ]:
# get the con_idxs and cat_idxs

cont_idxs = [i for i in range(X_train.shape[1]) if i not in cat_idxs]
print(cont_idxs)


In [ ]:
train_mean, train_std = np.array(X_train_d['data'][:,cont_idxs],dtype=np.float32).mean(0), np.array(X_train_d['data'][:,cont_idxs],dtype=np.float32).std(0)


In [21]:
print("cat_dims=", tuple(cat_dims) )
print("num_continuous =", len(cont_idxs))
print("dim =", opt.embedding_size)
print("dim_out =", 1)
print("depth =", opt.transformer_depth)
print("heads =", opt.attention_heads)
print("attn_dropout =", opt.attention_dropout)
print("ff_dropout =", opt.ff_dropout)
print("mlp_hidden_mults =", (4, 2))
print("cont_embeddings =", opt.cont_embeddings)
print("attentiontype =", opt.attentiontype)
print("final_mlp_style =", opt.final_mlp_style)
print("y_dim =", y_dim)

cat_dims= (1115, 7, 2, 4, 2, 4, 3, 2)
num_continuous = 9
dim = 32
dim_out = 1
depth = 1
heads = 4
attn_dropout = 0.8
ff_dropout = 0.8
mlp_hidden_mults = (4, 2)
cont_embeddings = MLP
attentiontype = colrow
final_mlp_style = sep
y_dim = 1


In [ ]:
print(opt)
model = SAINT(categories = tuple(cat_dims), 
                num_continuous = len(cont_idxs),                
                dim = opt.embedding_size,                           
                dim_out = 1,                       
                depth = opt.transformer_depth,                       
                heads = opt.attention_heads,                         
                attn_dropout = opt.attention_dropout,             
                ff_dropout = opt.ff_dropout,                  
                mlp_hidden_mults = (4, 2),       
                cont_embeddings = opt.cont_embeddings,
                attentiontype = opt.attentiontype,
                final_mlp_style = opt.final_mlp_style,
                y_dim = y_dim
                )

In [ ]:
criterion = nn.MSELoss().to(device)
model.to(device)


In [ ]:
## Choosing the optimizer
import torch.optim as optim


if opt.optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=opt.lr,
                          momentum=0.9, weight_decay=5e-4)
    from utils import get_scheduler
    scheduler = get_scheduler(opt, optimizer)
elif opt.optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(),lr=opt.lr)
elif opt.optimizer == 'AdamW':
    optimizer = optim.AdamW(model.parameters(),lr=opt.lr)
best_valid_auroc = 0
best_valid_accuracy = 0
best_test_auroc = 0
best_test_accuracy = 0
best_valid_rmse = 100000
print('Using the optimizer: ', opt.optimizer)
print('Training begins now.')

In [ ]:
from torch.utils.data import DataLoader

from data_openml import DataSetCatCon

continuous_mean_std = np.array([train_mean, train_std]).astype(np.float32) 

train_ds = DataSetCatCon(X_train_d, y_train_d, cat_idxs, task='regression', continuous_mean_std=continuous_mean_std)
trainloader = DataLoader(train_ds, batch_size=opt.batchsize, shuffle=True)

valid_ds = DataSetCatCon(X_valid_d, y_valid_d, cat_idxs, task='regression', continuous_mean_std=continuous_mean_std)
validloader = DataLoader(valid_ds, batch_size=opt.batchsize, shuffle=False)

test_ds = DataSetCatCon(X_test_d, y_test_d, cat_idxs, task='regression', continuous_mean_std=continuous_mean_std)
testloader = DataLoader(test_ds, batch_size=opt.batchsize, shuffle=False)

In [ ]:
from augmentations import embed_data_mask
from utils import count_parameters, classification_scores, mean_sq_error
import os

modelsave_path = os.path.join(opt.savemodelroot,opt.run_name)

for epoch in range(opt.epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # import ipdb; ipdb.set_trace()
        optimizer.zero_grad()
        # x_categ is the the categorical data, x_cont has continuous data, y_gts has ground truth ys. cat_mask is an array of ones same shape as x_categ and an additional column(corresponding to CLS token) set to 0s. con_mask is an array of ones same shape as x_cont. 
        x_categ, x_cont, y_gts, cat_mask, con_mask = data[0].to(device), data[1].to(device),data[2].to(device),data[3].to(device),data[4].to(device)

        # We are converting the data to embeddings in the next step
        _ , x_categ_enc, x_cont_enc = embed_data_mask(x_categ, x_cont, cat_mask, con_mask, model)           
        reps = model.transformer(x_categ_enc, x_cont_enc)
        # select only the representations corresponding to CLS token and apply mlp on it in the next step to get the predictions.
        y_reps = reps[:,0,:]
        
        y_outs = model.mlpfory(y_reps)
        if opt.task == 'regression':
            loss = criterion(y_outs,y_gts) 
        else:
            loss = criterion(y_outs,y_gts.squeeze()) 
        loss.backward()
        optimizer.step()
        if opt.optimizer == 'SGD':
            scheduler.step()
        running_loss += loss.item()
    # print(running_loss)
    if opt.active_log:
        wandb.log({'epoch': epoch ,'train_epoch_loss': running_loss, 
        'loss': loss.item()
        })
    if epoch%5==0:
            model.eval()
            with torch.no_grad():
                if opt.task in ['binary','multiclass']:
                    accuracy, auroc = classification_scores(model, validloader, device)
                    test_accuracy, test_auroc = classification_scores(model, testloader, device)

                    print('[EPOCH %d] VALID ACCURACY: %.3f, VALID AUROC: %.3f' %
                        (epoch + 1, accuracy,auroc ))
                    print('[EPOCH %d] TEST ACCURACY: %.3f, TEST AUROC: %.3f' %
                        (epoch + 1, test_accuracy,test_auroc ))
                    if opt.active_log:
                        wandb.log({'valid_accuracy': accuracy ,'valid_auroc': auroc })     
                        wandb.log({'test_accuracy': test_accuracy ,'test_auroc': test_auroc })  
                    if opt.task =='multiclass':
                        if accuracy > best_valid_accuracy:
                            best_valid_accuracy = accuracy
                            best_test_auroc = test_auroc
                            best_test_accuracy = test_accuracy
                            torch.save(model.state_dict(),'%s/bestmodel.pth' % (modelsave_path))
                    else:
                        if accuracy > best_valid_accuracy:
                            best_valid_accuracy = accuracy
                        # if auroc > best_valid_auroc:
                        #     best_valid_auroc = auroc
                            best_test_auroc = test_auroc
                            best_test_accuracy = test_accuracy               
                            torch.save(model.state_dict(),'%s/bestmodel.pth' % (modelsave_path))

                else:
                    valid_rmse = mean_sq_error(model, validloader, device)    
                    test_rmse = mean_sq_error(model, testloader, device)  
                    print('[EPOCH %d] VALID RMSE: %.3f' %
                        (epoch + 1, valid_rmse ))
                    print('[EPOCH %d] TEST RMSE: %.3f' %
                        (epoch + 1, test_rmse ))
                    if opt.active_log:
                        wandb.log({'valid_rmse': valid_rmse ,'test_rmse': test_rmse })     
                    if valid_rmse < best_valid_rmse:
                        best_valid_rmse = valid_rmse
                        best_test_rmse = test_rmse
                        torch.save(model.state_dict(),'%s/bestmodel.pth' % (modelsave_path))
            model.train()